In [ ]:
import numpy as np
from numpy import loadtxt
from keras.models import Sequential # tipo de modelo
from keras.layers import Dense #camadas no caso utilizamos a do tipo densa
from sklearn.model_selection import train_test_split # separar um conjunto de testes
from sklearn.metrics import confusion_matrix
from keras.layers import Dropout
from sklearn import metrics
import matplotlib.pyplot as plt

#Dataset Retirado de https://www.kaggle.com/rishanmascarenhas/covid19-temperatureoxygenpulse-rate
#Codigo realizado no GoogleCollab
#Foi feito uma conversão de resultados ou seja, Negative = 0, Positive = 1
#Dataset com os valores ajustados se encontra neste link: https://github.com/salom100/IA/blob/ab284e3a07fffa880b9988b50ddb567a69c0c0fc/Redes%20Neurais%20Atividades/Corona1.csv
#ID, Oxigenio, Pulso, Temperatura (F), Resultado/Diagnostico
#Carregar os dados
dataset = loadtxt('/content/covid/Corona1.csv', delimiter=',')
X = dataset[:,1:4]
y = dataset[:, 4]

#Normalização
X = (X-np.min(X)/np.max(X)-np.min(X))

#shuffle - ele embaralha os dados
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=42, shuffle=True)

#X = (X-np.min(X)/(np.max(X)-np.min(X))

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


#Definindo modelo
model = Sequential()
model.add(Dense(100, input_dim=X.shape[1], activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(33, activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(11, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#Compilando modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Modelo de Fit
history = model.fit(X_train,y_train, validation_data=(X_test,y_test), epochs=200, batch_size=128)

#Grafico de desempenho da accuracy
# A acuracia é muito fraca para se usar como medida determinante
print(history.history.keys())
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#Grafico de desempenho do loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Evaluação do modelo
#_, accuracy = model.evaluate(X_test,y_test)
#print('Accuracy: %.2f' % (accuracy * 100))

pred = model.predict_classes(X_test)
cm = confusion_matrix(y_test, pred)

#print(pred)
#print(cm)

#VN,FP,FN,VP
tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
tpr = tp/(tp+fn) # taxa do acerto positivo
tnr = tn/(tn+fp) # Taxa do acerto negativo
acc = (tp+tn)/(tp+tn+fn+fp)

print("TPR: ",tpr)
print("TNR: ",tnr)
print("ACC: ",acc)

#AUC Área abaixo da Curva (curva de roc)
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred, pos_label=1)
auc = metrics.auc(fpr,tpr)

#Grafico da curva de ROC
plt.plot([0,1],[0,1],linestyle='--', lw=2, color='r', label='Chance', alpha=.8)
plt.plot(fpr,tpr, color = 'b', label=r'ROC ( AUC = %0.2f)' % (auc), lw=2 , alpha=.8)
plt.suptitle("ROC Curve")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.legend(loc = "lower right")
plt.show()
